In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from tensorflow.keras import  models, optimizers, layers, activations
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import History
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications import ResNet50
from keras.applications.xception import Xception


ImportError: ignored

In [2]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D ,GlobalAveragePooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import BatchNormalization

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11043068144230994514
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11320098816
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13419754637859997689
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
xla_global_id: 416903419
]


In [4]:
from google.colab import drive
drive.mount('/content/drive')
path = "drive/MyDrive/inaturalist_12K"

Mounted at /content/drive


In [5]:
image_size=(256,256,3)
output_size=10

In [6]:
def get_data(path,augmentation=True):
    train_path=os.path.join(path,"train")
    test_path=os.path.join(path,"val")
    if augmentation==False:
        train_generator=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split=0.1)
    else:
        train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                          rotation_range=90,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          validation_split=0.1,
                                          horizontal_flip=True)
    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    train_data = train_generator.flow_from_directory(
    directory=train_path,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42)
    valid_data=train_generator.flow_from_directory(
    directory=train_path,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42)
    test_data=test_generator.flow_from_directory(
    directory=test_path,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42)
    return train_data,valid_data,test_data

In [7]:
model_b=False

Inception REsnet V2

In [10]:
class inc_v2(object):
  def __init__(self,input_size,output_size,drop_out,batch_normalization,dense_size):
    self.input_size =input_size
    self.output_size=output_size
    self.drop_out=drop_out
    self.batch_normalization=batch_normalization
    self.initialize(input_size,output_size,drop_out,batch_normalization,dense_size)
  def initialize(self,input_size,output_size,drop_out,batch_normalization,dense_size):
    model_b=InceptionResNetV2(include_top=False,
            weights="imagenet",
            input_shape=input_size,
        )
    model_b.trainable=False
    input = keras.Input(shape=input_size)
    x = model_b(input, training=False)
    self.model=Sequential()
    self.model.add(Model(input, x))
    self.model.add(Flatten())
    self.model.add(Dense(dense_size))
    self.model.add(Activation('selu'))
    if batch_normalization==True:
        self.model.add(BatchNormalization())
    self.model.add(Dropout(drop_out))
    self.model.add(Dense(output_size))
    self.model.add(Activation("softmax"))



INCEPtion V_3

In [11]:
class inc_v3(object):
  def __init__(self,input_size,output_size,drop_out,batch_normalization,dense_size):
    self.input_size =input_size
    self.output_size=output_size
    self.drop_out=drop_out
    self.batch_normalization=batch_normalization
    self.initialize(input_size,output_size,drop_out,batch_normalization,dense_size)
  def initialize(self,input_size,output_size,drop_out,batch_normalization,dense_size):
    model_b=InceptionV3(include_top=False,
            weights="imagenet", 
            input_shape=input_size,
        )
    model_b.trainable=False
    input = keras.Input(shape=input_size)
    x = model_b(input, training=False)
    self.model=Sequential()
    self.model.add(Model(input, x))
    self.model.add(Flatten())
    self.model.add(Dense(dense_size))
    self.model.add(Activation('selu'))
    if batch_normalization==True:
        self.model.add(BatchNormalization())
    self.model.add(Dropout(drop_out))
    self.model.add(Dense(output_size))
    self.model.add(Activation("softmax"))

Xception

In [12]:
class Xcpt_ion(object):
  def __init__(self,input_size,output_size,drop_out,batch_normalization,dense_size):
    self.input_size =input_size
    self.output_size=output_size
    self.drop_out=drop_out
    self.batch_normalization=batch_normalization
    self.initialize(input_size,output_size,drop_out,batch_normalization,dense_size)
  def initialize(self,input_size,output_size,drop_out,batch_normalization,dense_size):
    model_b=Xception(include_top=False,
            weights="imagenet",
            input_shape=input_size)
    model_b.trainable=False
    input = keras.Input(shape=input_size)
    x = model_b(input, training=False)
    self.model=Sequential()
    self.model.add(Model(input, x))
    self.model.add(Flatten())
    self.model.add(Dense(dense_size))
    self.model.add(Activation('selu'))
    if batch_normalization==True:
        self.model.add(BatchNormalization())
    self.model.add(Dropout(drop_out))
    self.model.add(Dense(output_size))
    self.model.add(Activation("softmax"))

In [17]:
model=Xcpt_ion(image_size,output_size,0.4,True,32)
model.model.summary()

83697664/83683744 [==============================] - 0s 0us/step
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_2 (Functional)        (None, 8, 8, 2048)        20861480  
                                                                 
 flatten_2 (Flatten)         (None, 131072)            0         
                                                                 
 dense_4 (Dense)             (None, 32)                4194336   
                                                                 
 activation_192 (Activation)  (None, 32)               0         
                                                                 
 batch_normalization_194 (Ba  (None, 32)               128       
 tchNormalization)                                               
                                                                 
 dropout_2 (Dropout)         (None, 32)                

In [14]:
trained_models={"inception_v2":inc_v2,"inception_v3":inc_v3,"Xception":Xcpt_ion}
!pip install wandb

In [15]:
import wandb
from wandb.keras import WandbCallback

In [16]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [17]:
import numpy as np
from tensorflow.keras.optimizers import Adam
np.random.seed(2)
class Runner(object):
    def __init__(self,inaturalist_data=True,augmentation=True):
        if inaturalist_data:
            self.initialize_inaturalist_data(augmentation)
        else:
            self.initialize_data()

    def initialize_inaturalist_data(self,augmentation=True):
        self.train_data,self.valid_data,self.test_data= get_data(path,augmentation)
    

    def initialize_data(self):
        raise NotImplementedError("Please implement this method if you need other dataset.")

    @staticmethod
    def get_activation_function(key):
        mapper = {
      #"sigmoid": Sigmoid,
      "relu": 'relu'
      #"tanh": Tanh
    }
        assert key in mapper
        return mapper[key]

def run_wandb():
    wandb.init(project="pretrained_model", entity="cs21m003_cs21d406")
    config = wandb.config
    base_model=trained_models[config.models]
    wandb.run.name=f"model{config.models}_e_{config.epochs}_bs_{config.batch_size}_rate_{config.learning_rate}_aug_{config.augmentation}_BN_{config.batch_normalization}_drp_{config.drop_out}_dense_{config.dense_size}"
    runner=Runner(True,config.augmentation)
    """
    params = {
    "epochs"        : config.epochs,
    "batch_size"    : config.batch_size,
    "act_func"      : config.act_func,
    "learning_rate" : config.learning_rate,
    "augmentation"  : config.augmentation,
    "batch_normalization"  : config.batch_normalization,
    "drop_out"  : config.drop_out,
    "dense_size":config.dense_size,
    "models": config.models

    }
    """
    model_1=base_model(image_size,output_size,config.drop_out,config.batch_normalization,config.dense_size)
    model_1.model.compile(
    optimizer=Adam(config.learning_rate),  # Optimizer
    loss="categorical_crossentropy", metrics="categorical_accuracy")
    model_1.model.fit(runner.train_data,epochs=config.epochs,batch_size=config.batch_size,validation_data=runner.valid_data,
          callbacks=[WandbCallback()])
    loss,accuracy=model_1.model.evaluate(runner.test_data,batch_size=config.batch_size)
    model_b.trainable=True
    print(f'test accuracy:{accuracy}')
    wandb.log({"test accuracy":accuracy})

def do_hyperparameter_search_using_wandb():
    sweep_config = {
    "name": "random sweep",
    "method": "random",
    "metric":{
      "name": "ValidationAccuracy",
      "goal": "maximize"
    },
    "parameters":{
      "epochs": {"values": [5, 10]}, 
      "batch_size": {"values": [64]}, 
      #"act_func": {"values": ['elu', 'relu', 'selu']}, 
      "learning_rate": {"values": [1e-3, 1e-4]}, 
      "augmentation": {"values": [True,False]} , 
      "batch_normalization": {"values": [True,False]},
      "models": {"values": ["inception_v2","inception_v3","Xception"]},
      "drop_out": {"values": [0.3,0.4,0.5]},
      "dense_size": {"values": [32,64]}}}
  
    sweep_id = wandb.sweep(sweep_config, project = "pretrained_model",entity='cs21m003_cs21d406')
    wandb.agent(sweep_id, function=run_wandb,count=1)

In [ ]:
do_hyperparameter_search_using_wandb()

Create sweep with ID: m7zxiuz4
Sweep URL: https://wandb.ai/cs21m003_cs21d406/pretrained_model/sweeps/m7zxiuz4


wandb: Agent Starting Run: isyvfdxh with config:
wandb: 	augmentation: True
wandb: 	batch_normalization: True
wandb: 	batch_size: 64
wandb: 	dense_size: 64
wandb: 	drop_out: 0.5
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	models: inception_v3
wandb: Currently logged in as: akashsainics21m003 (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Found 10005 images belonging to 10 classes.
Found 10005 images belonging to 10 classes.
Found 2019 images belonging to 10 classes.
87924736/87910968 [==============================] - 1s 0us/step
Epoch 1/5
268/313 [========================>.....] - ETA: 5:56 - loss: 1.3061 - categorical_accuracy: 0.6005

In [ ]:
train_data,valid_data,test_data= get_data(path,True)
model.model.compile(
optimizer=keras.optimizers.Adam(0.001),  # Optimizer
loss="categorical_crossentropy", metrics="categorical_accuracy")
  #train_data,valid_data,test_data=get_data(path,augmentation=True)
model.model.fit(train_data,epochs=1,batch_size=32,validation_data=valid_data)
loss,accuracy=model.model.evaluate(test_data,batch_size=64)
print(loss)
print(accuracy)